In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import pandas as pd

# Load your data
data = pd.read_csv('train.csv')

# Preprocess the text
data['cleaned_text'] = data['abstract_text'].str.lower()  # Example preprocessing

# Tokenization and Padding Parameters
vocab_size = 60000
max_length = 64
trunc_type = 'post'
padding_type = 'post'
oov_tok = "<OOV>"

# Tokenization
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(data['cleaned_text'])
word_index = tokenizer.word_index

# Convert to sequences and pad
sequences = tokenizer.texts_to_sequences(data['cleaned_text'])
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

# Label Encoding
labels = pd.get_dummies(data['target']).values

# Splitting the data128
train_sequences, validation_sequences, train_labels, validation_labels = train_test_split(padded_sequences, labels, test_size=0.1)

# Building the LSTM Model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 128, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(len(data['target'].unique()), activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

# Training the LSTM Model
history = model.fit(train_sequences, train_labels, epochs=10, validation_data=(validation_sequences, validation_labels))





Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 128, 128)          7680000   
                                                                 
 bidirectional (Bidirection  (None, 128)               98816     
 al)                                                             
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dense_1 (Dense)             (None, 5)                 325       
                                                                 
Total params: 7787397 (29.71 MB)
Trainable params: 7787397 (29.71 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10


2250/2250 [==============================] - 172s 75ms/step - loss